In [2]:
"""
Model IO
- 영화 이름을 가지고 감독, 주요 출연진, 예산, 흥행 수익, 영화의 장르, 간단한 시놉시스 등 영화에 대한 정보로 답장하는 체인을 만드세요.
- LLM은 **항상** 동일한 형식을 사용하여 응답해야 하며, 이를 위해서는 원하는 출력의 예시를 LLM에 제공해야 합니다.
- 예제를 제공하려면 `FewShotPromptTemplate` 또는 `FewShotChatMessagePromptTemplate`을 사용하세요.
"""

# LangChain의 FewShotPromptTemplate, ChatPromptTemplate, ChatOpenAI 등 주요 구성요소 임포트
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

# 1. 예시 데이터 준비 (Few-shot learning: LLM이 따라야 할 답변 포맷을 명확히 예시로 제공)
# 실제로는 DB 등에서 불러올 수도 있음
examples = [
    {
        "movie": "Inception",
        "info": """감독: 크리스토퍼 놀란
주요 출연진: 레오나르도 디카프리오, 조셉 고든 레빗, 엘렌 페이지
예산: $160,000,000
흥행 수익: $829,895,144
장르: SF, 액션, 스릴러
시놉시스: 꿈을 조작하는 기술을 이용해 타인의 무의식에 침투하는 산업 스파이의 이야기"""
    },
    {
        "movie": "Parasite",
        "info": """감독: 봉준호
주요 출연진: 송강호, 이선균, 조여정
예산: $11,400,000
흥행 수익: $263,000,000
장르: 드라마, 스릴러
시놉시스: 가난한 가족이 부잣집에 점차 스며들며 벌어지는 블랙코미디"""
    }
]

# 2. 예시 포맷 정의 (ChatPromptTemplate의 from_messages 사용)
# - Human: 영화 이름을 묻는 질문
# - AI: 영화 정보 답변
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "영화 '{movie}'에 대해 알려줘."),
    ("ai", "{info}")
])

# 3. FewShotChatMessagePromptTemplate로 예시와 포맷 지정
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

# 4. 최종 프롬프트 구성 (시스템 메시지 + 예시 + 사용자 질문)
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 정보 전문가입니다. 항상 동일한 형식으로 답변하세요."),
    few_shot_prompt,
    ("human", "영화 '{movie}'에 대해 알려줘.")
])

# 5. LLM 모델 준비 (ChatOpenAI 등)
chat = ChatOpenAI(temperature=0.1)

# 6. 체인 구성 (프롬프트 → LLM)
chain = final_prompt | chat

# 7. 실제 호출 예시
result = chain.invoke({"movie": "명량"})
print(result)

# ---------------------------
# 주요 주석 요약
# - FewShotChatMessagePromptTemplate: 예시 기반 프롬프트 생성, LLM이 예시 패턴을 따라 답변하도록 유도
# - ChatPromptTemplate: 메시지 기반 프롬프트, 시스템/예시/사용자 질문 조합
# - 체인(LCEL): 프롬프트와 LLM을 연결해 일관된 답변 생성
# - 공식 문서 및 강의에서 강조한 '예시 기반 프롬프트 설계', '동일 포맷 강제', '시스템 메시지 활용' 등 반영
# ---------------------------

content='감독: 김한민\n주요 출연진: 최민식, 조진웅, 류승룡\n예산: ₩25,000,000,000\n흥행 수익: ₩135,700,000,000\n장르: 역사, 전쟁, 액션\n시놉시스: 조선시대 임진왜란 당시 조선 해군의 최대 해전인 명량 해전을 다룬 역사 영화'
